In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip uninstall opencv-python -y

In [ ]:
!pip install opencv-python

In [ ]:
!pip install PyPDF2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import PyPDF2
import cv2
import numpy as np
import os

In [ ]:
df = pd.read_csv("/kaggle/input/data-visualization/Data Visualization/docs_nums.csv")

In [ ]:
df_form = df[df['type']=="Form 1040"]

In [ ]:
BASE_PDF_DIR = "/kaggle/input/data-visualization/Data Visualization/Files"
BASE_IMG_DIR = "/kaggle/input/data-visualization/Data Visualization/Images"

In [ ]:
name = df_form.iloc[0]['filename'].split(".")[0] + ".pdf"
PDF_PATH = f"{BASE_PDF_DIR}/{name}"
img_name = df_form.iloc[0]['filename']
IMG_PATH = f'{BASE_IMG_DIR}/{img_name}'
print(IMG_PATH, PDF_PATH)

In [ ]:
PDF_PATH = f"{BASE_PDF_DIR}/{name}"
with  open(PDF_PATH, 'rb') as pdfFileObj:
    
    pdfReader = PyPDF2.PdfReader(pdfFileObj)
    pageObj = pdfReader.pages[0]


    # print(pdfReader.get_form_text_fields())
    # print(pdfReader.get_fields())
    
    text = pageObj.extract_text()
    # print(text[4276:])
    name = text[4276:].split("\n")[0] #contains first and last name
    
    fields = text[4276:].split("\n")[1:]#contain income details

In [ ]:
sabai = {}
field_names = ['Basic Information___First Name', 'Basic Information___Last Name', 'Income___One_A','Income___One_B','Income___One_C','Income___One_D',
 'Income___One_E','Income___One_F','Income___One_G','Income___One_H','None', 'Income___One_Z','Income___Two_B',
 'Income___Three_B', 'Income___Four_B', 'Income___Five_B',
 'Income___Six_B','None','Income___Seven' , 'Income___Eight','Income___Nine', 'Income___Ten', 'Income___Eleven',
 'Income___Twelve', 'Income___Thirteen', 'Income___Fourteen','Income___Fifteen'
   ]
for name in field_names:
    sabai[name] = []

In [ ]:
field_names = ['Basic Information___First Name', 'Basic Information___Last Name', 'Income___One_A','Income___One_B','Income___One_C','Income___One_D',
 'Income___One_E','Income___One_F','Income___One_G','Income___One_H','None', 'Income___One_Z','Income___Two_B',
 'Income___Three_B', 'Income___Four_B', 'Income___Five_B',
 'Income___Six_B','None','Income___Seven' , 'Income___Eight','Income___Nine', 'Income___Ten', 'Income___Eleven',
 'Income___Twelve', 'Income___Thirteen', 'Income___Fourteen','Income___Fifteen'
   ]

def get_numeric_data(IMG_PATH, PDF_PATH):
    d = {}
    
    with  open(PDF_PATH, 'rb') as pdfFileObj:
    
        pdfReader = PyPDF2.PdfReader(pdfFileObj)
        pageObj = pdfReader.pages[0]


        # print(pdfReader.get_form_text_fields())
        # print(pdfReader.get_fields())

        text = pageObj.extract_text()
        # print(text[4276:])
        name = text[4276:].split("\n")[0] #contains first and last name

        fields = text[4276:].split("\n")[1:]#contain income details
    
    img2 = cv2.imread(IMG_PATH,cv2.IMREAD_GRAYSCALE)
    img2 = cv2.resize(img2,(1698,2198))


    img = cv2.imread(IMG_PATH)
    img = cv2.resize(img,(1698,2198))

    img_h,img_w = img.shape[:2]

    (thresh, img2) = cv2.threshold(img2, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    cv2.rectangle(img,(int(111),int(330)),(int(640),int(361)),(0,0,255),2)
    cv2.rectangle(img,(int(673.3),int(325.9)),(int(1258.6),int(359.5)),(0,0,255),2)


    # create a window
    # cv2.namedWindow('Point Coordinates')

    cropped_img = img2[int(1017.7):int(1848.5),int(1405.7/1698*img_w):int(1596/1698*img_w)]
    cropped_shape = cropped_img.shape

    height = cropped_shape[0]//25
    width = cropped_shape[1]
    ind = 0
    for i,f_name in enumerate(field_names[2:]):
        if i ==8 or i==15:
            continue
        cut = cropped_img[i*height:(i+1)*height,:] 
        mean = np.mean(cut[:,int(width*6/14):int(width*8/14)])
        # print(mean)
        if mean < 230:
    #         print(f'{f_name}:{fields[ind]} bbox: {(int(1405.7),int(1017.7)+i*height),(int(1405.7)+width,int(1017.7)+(i+1)*height)}')
            sabai[f'{f_name}'].append(fields[ind])
            cv2.rectangle(img,(int(1405.7),int(1017.7)+i*height),(int(1405.7)+width,int(1017.7)+(i+1)*height),(0,0,255),2)
            ind+=1
        else:
            sabai[f'{f_name}'].append(None)
    return d

In [ ]:
files = df_form["filename"]

In [ ]:
datas = []

In [ ]:
for file in files:
    pdfname = file.split(".")[0] + ".pdf"
    PDF_PATH = f"{BASE_PDF_DIR}/{pdfname}"
    IMG_PATH = f"{BASE_IMG_DIR}/{file}"
    datas.append(get_numeric_data(IMG_PATH, PDF_PATH))

In [ ]:
sabai['Income___One_A'])

In [ ]:
final_df = df_form.copy()

In [ ]:
final_df['Income___One_A'] = sabai['Income___One_A']

In [ ]:
sabai
del sabai['Basic Information___First Name']
del sabai['Basic Information___Last Name']

In [ ]:
del sabai['None']

In [ ]:
for field in sabai.keys():
    final_df[field] = sabai[field]

In [ ]:
final_df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
final_df.to_csv("form_1040_viz.csv", index=False)

In [ ]:
mapping = {
    'Income___One_A': "Total Income From Employer",
    "Income___Nine": "Total Income",
    "Income___Eleven": "Adjusted Gross Income",
    "Income___Fifteen": "Taxable Income"
}

In [ ]:
final_df.columns

In [ ]:
final_df['Income___Fifteen'].to_list()

In [ ]:
import matplotlib.pyplot as plt

def plot(column):
    # Given data (replace this with your data)
    data = final_df[f'{column}'].to_list()
    # Filter out None values
    data_filtered = [val for val in data if val is not None]
    data_filtered = [float(val) for val in data_filtered]
    # Creating the boxplot
    plt.figure(figsize=(8, 6))
    plt.boxplot(data_filtered, vert=False)
    plt.title(f'Boxplot of {mapping[column]}')
    plt.xlabel(f'{mapping[column]}')
    plt.show()

In [ ]:
plot("Income___One_A")

In [ ]:
plot("Income___Nine")

In [ ]:
plot("Income___Eleven")

In [ ]:
import numpy as np

def histo(column):
    data = final_df[f'{column}'].to_list()

    # Convert numeric strings to floats and filter out non-numeric values and None
    data_filtered = [float(val) for val in data if val is not None]

    # Create histogram
    plt.figure(figsize=(8, 6))
    plt.hist(data_filtered, bins=10, edgecolor='black')  # Adjust the number of bins as needed
    plt.title(f'Histogram of {mapping[column]}')
    plt.xlabel(f'{mapping[column]}')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

In [ ]:
mapping = {
    'Income___One_A': "Total Income From Employer",
    "Income___Nine": "Total Income",
    "Income___Eleven": "Adjusted Gross Income",
    "Income___Fifteen": "Taxable Income"
}

In [ ]:
histo("Income___Nine")
histo("Income___One_A")
histo("Income___Eleven")
histo("Income___Fifteen")